# Demo caching

This notebook shows how caching of daily results is organised. First we show the low-level approach, then a high-level function is used.

## Low-level approach

In [1]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
from opengrid.library import analysis
import charts
hp = houseprint.Houseprint()

Server running in the folder /mnt/data/work/opengrid/code/opengrid/notebooks at 127.0.0.1:42324
Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
23 Sites created
23 Devices created
73 sensors created
Houseprint parsing complete


We demonstrate the caching for the minimal daily water consumption (should be close to zero unless there is a water leak).  We create a cache object by specifying what we like to store and retrieve through this object.  The cached data is saved as a single csv per sensor in a folder specified in the opengrid.cfg.  Add the path to a folder where you want these csv-files to be stored as follows to your opengrid.cfg

    [data]
    folder: path_to_folder

In [3]:
cache_standby = caching.Cache(variable='elec_standby_night')
df_cache = cache_standby.get(sensors=hp.get_sensors(sensortype='elec'))
charts.plot(df_cache.ix[-8:], stock=True, show='inline')

Cache object created for variable: elec_standby_night


ValueError: No objects to concatenate

If this is the first time you run this demo, no cached data will be found, and you get an empty graph. 

Let's store some results in this cache.  We start from the water consumption of last week.

In [ ]:
hp.sync_tmpos()

In [ ]:
start = pd.Timestamp('now') - pd.Timedelta(weeks=1)
df_water = hp.get_data(sensortype='water', head=start )
df_water.info()

We use the method *daily_min()* from the analysis module to obtain a dataframe with daily minima for each sensor.

In [ ]:
daily_min = analysis.daily_min(df_water)
daily_min.info()

In [ ]:
daily_min

In [ ]:
cache_water.update(daily_min)

Now we can get the daily water minima from the cache directly. Pass a *start* or *end* date to limit the returned dataframe.

In [ ]:
sensors = hp.get_sensors(sensortype='water') # sensor objects
charts.plot(cache_water.get(sensors=sensors, start=start, end=None), show='inline', stock=True)

## A high-level cache function

The caching of daily results is very similar for all kinds of results.  Therefore, a high-level function is defined that can be parametrised to cache a lot of different things.

In [ ]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
from opengrid.library import analysis
import charts   

In [ ]:
hp = houseprint.Houseprint()
#hp.sync_tmpos()

In [ ]:
sensors = hp.get_sensors(sensortype='water')
caching.cache_results(hp=hp, sensors=sensors, function='daily_min', resultname='water_daily_min')

In [ ]:
cache = caching.Cache('water_daily_min')
daily_min = cache.get(sensors = sensors, start = '20151201')
charts.plot(daily_min, stock=True, show='inline')